In [ ]:
#Practica 4 introduccion a la dinamica molecular
#Tinoco Videgaray Sergio Ernesto
#Bioinformatics
#21/03/24

In [ ]:
import cv2
import numpy as np
from IPython import display as display
import ipywidgets as ipw
from PIL import  Image
from io import BytesIO
import numpy as np

In [ ]:
class Atomo():  #Clase atomo
    def __init__(self,q,r,v,a):
        self.q=q    #Carga
        self.r=r    #Posicion
        self.v=v    #Velocidad
        self.a=a    #Aceleracion

In [ ]:
class MoleculaAgua():   #Clase Molecula de agua
    def __init__(self,q,r,v,f,angle):
        self.atomos=[]  #Arreglo atomos
        self.atomos.append(Atomo(q[0],r,v,f))
        self.atomos.append(Atomo(q[1],(r[0]+50,r[1]-50),v,f))
        self.atomos.append(Atomo(q[2],(r[0]-50,r[1]-50),v,f))
        self.oxigeno=self.atomos[0] if q[0]<0 else self.atomos[1] if q[1]<0 else self.atomos[2]
        self.angle=angle    #Angulo de flexion


    def getAtomos(self):    #Devolver lista de atomos
        return self.atomos

In [ ]:
#Funciones de movimiento
def new_r(r,v,a,dt):  #Actualizar posicion
  rx,ry=r
  vx,vy=v
  ax,ay=a

  new_rx = rx + vx*dt + 0.5*ax*(dt**2)
  new_ry = ry + vy*dt + 0.5*ay*(dt**2)
  return new_rx, new_ry

def new_v(v,a,dt):  #Actualizar velocidad
  vx,vy=v
  ax,ay=a

  new_vx = vx + ax*dt
  new_vy = vy + ay*dt
  return new_vx,new_vy

def new_a(f):   #Actualizar aceleracion
  fx,fy=f
  #Masa unitaria
  new_ax =  fx/1
  new_ay =  fy/1
  return [new_ax,new_ay]

def Fc(a,b,qA,qB):  #Actualizar fuerza entre cargas
   ax,ay=a
   bx,by=b
   ABx,ABy=bx-ax,by-ay
   magAB=np.sqrt(ABx**2+ABy**2)
   nABx,nABy=ABx/magAB,ABy/magAB
   k=-90
   magF=k*(qA*qB)/(magAB**2)
   return nABx*magF,nABy*magF

def Fs(a,b,d_eq,k): #Actualizar fuerza elastica de los enlaces
  ax,ay=a
  bx,by=b
  ABx,ABy=bx-ax,by-ay
  magAB=np.sqrt(ABx**2+ABy**2)
  nABx,nABy=ABx/magAB,ABy/magAB
  magF=-k*(magAB-d_eq)
  return nABx*magF,nABy*magF

def getAngle(A,B,C):  #Obtener angulo de flexion
  BA=np.subtract(A,B)
  BC=np.subtract(C,B)
  magBA=np.linalg.norm(BA)
  magBC=np.linalg.norm(BC)
  return np.degrees(np.arccos(np.dot(BA,BC)/(magBA*magBC)))

def U(A,B,C,k,a_eq,molecula): #Energia de potencial de flexion
  global img
  molecula.angle=getAngle(A,B,C)
  return k*((molecula.angle-a_eq)**2)

def gradU(A,B,C,k,a_eq,atomo_mover,molecula): #Gradiente de flexion
  D=0.01
  u=U(A,B,C,k,a_eq,molecula)
  u_dx=u_dy=u
  if atomo_mover==A:      #Mover A
    u_dx=U((A[0]+D,A[1]),B,C,k,a_eq,molecula)
    u_dy=U((A[0],A[1]+D),B,C,k,a_eq,molecula)
  elif atomo_mover == B:  #Mover B
    u_dx=U(A,(B[0]+D,B[1]),C,k,a_eq,molecula)
    u_dy=U(A,(B[0],B[1]+D),C,k,a_eq,molecula)
  elif atomo_mover==C:    #Mover C
    u_dx=U(A,B,(C[0]+D,C[1]),k,a_eq,molecula)
    u_dy=U(A,B,(C[0],C[1]+D),k,a_eq,molecula)

  return (u_dx-u)/D,(u_dy-u)/D

In [ ]:
def mostrarMoleculas(moleculas,wIm): #Graficar moleculas
    global tam,img
    x=y=tam//2
    for mol in moleculas:
      #Atomo oxigeno
      cv2.circle(img,(int(x+mol.oxigeno.r[0]),int(y-mol.oxigeno.r[1])),10,(255,0,0),-1)
      for atomo in mol.getAtomos():
        #Atomos hidrogeno
        if(atomo.q>0):  #Si tiene carga positiva
          #Generar linea del enlace
          cv2.line(img,(int(x+atomo.r[0]),int(y-atomo.r[1])),
                   (int(x+mol.oxigeno.r[0]),int(y-mol.oxigeno.r[1])) ,
                    (180,180,180), 3)
          #Atomo hidrogeno
          cv2.circle(img,(int(x+atomo.r[0]),int(y-atomo.r[1])),10,(255,255,255),-1)
        pilIm = Image.fromarray(img, mode="RGB")
    with BytesIO() as fOut: #Actualizar imagen
        pilIm.save(fOut, format="png")
        byPng = fOut.getvalue()
    wIm.value=byPng

In [ ]:
def update_Coulomb(moleculas):  #Actualizar fuerzas entre atomos
    global dt
    atomos=[]
    for molecula in moleculas:  #Por cada molecula
        for atomo in molecula.getAtomos():  #Por cada atomo
            atomos.append(atomo)    #Agregar a la lista de atomos
    for i in range(len(atomos)):
        fx=fy=0
        for j in range(len(atomos)):
            if atomos[i].r!=atomos[j].r:    #Atomos distintos
                #Aplicar ley de Coulomb
                fcx,fcy=Fc(atomos[i].r,atomos[j].r,atomos[i].q,atomos[j].q)
                fx+=fcx;fy+=fcy #Acumular fuerza total
        atomos[i].f=new_a((fx,fy))  #Actualizar fuerza/aceleracion
        atomos[i].v=new_v(atomos[i].v,atomos[i].f,dt)   #Actualizar velocidad
        atomos[i].r=new_r(atomos[i].r,atomos[i].v,atomos[i].f,dt)   #Actualizar posicion

In [ ]:
def update(moleculas,ka,a_eq):  #Actualizar moleculas
  global dt
  #Valores iniciales
  fsx=fsy=fx=fy=fuax=fuay=0
  angle_eq=a_eq
  d_eq=70
  update_Coulomb(moleculas) #Acualizar fuerzas entre cargas
  for molecula in moleculas:  #Por cada moleucla
    atomos=molecula.getAtomos()
    #Definir orden de atomos
    if atomos[0].q<0:
      A,B,C=atomos[2],atomos[0],atomos[1]
    elif atomos[1].q<0:
      A,B,C=atomos[0],atomos[1],atomos[2]
    else:
      A,B,C=atomos[1],atomos[2],atomos[0]
    for atomo in atomos:  #Por cada atomo
      #Spring
      if atomo!=B:  #Fuerza de rebote del enlace respecto al oxigeno
        fsx,fsy=Fs(B.r,atomo.r,d_eq,ka)
      else: #Fuerza de rebote respecto a hidrogenos
        fsAx,fsAy=Fs(A.r,B.r,d_eq,ka)
        fsBx,fsBy=Fs(C.r,B.r,d_eq,ka)
        fsx=fsAx+fsBx;fsy=fsAy+fsBy
      #Fuerza gradiente de flexion
      fuax,fuay=gradU(A.r,B.r,C.r,ka,angle_eq,atomo.r,molecula)
      #Fuerza total
      fx=(fsx-fuax);fy=(fsy-fuay)

      #Actualizar valores
      atomo.f=new_a((fx,fy))  #Actualizar fuerza
      atomo.v=new_v(atomo.v,atomo.f,dt) #Actualizar velocidad
      atomo.r=new_r(atomo.r,atomo.v,atomo.f,dt) #Actualizar posicion

In [ ]:
def main(qa,a_eq,ka):
    #Recibe nivel de carga, angulo de equilibrio y la constante k
    q=qa    #Cargas uniformes
    #Objetos molecula de agua
    moleculas=[]
    moleculas.append(MoleculaAgua((q,-q,q),(-40,60),(0,0),[0,0],0))
    moleculas.append(MoleculaAgua((q,q,-q),(60,-60),(0,0),[0,0],0))

    MaxIterations = 500
    global tam,dt,img
    tam=400
    dt=0.02
    img = np.zeros((tam, tam, 3), dtype="uint8")
    wIm = ipw.Image()
    display.display(wIm)

    for count in range(MaxIterations):  #Iterar
        img[:]=(0,0,0)
        #Numero de iteracion
        cv2.putText( img,str(count),(tam//3,25),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255))
        #Angulo de la molecula 1
        cv2.putText(img,str(int(moleculas[0].angle)),(30,55),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255))
        #Angulo de la molecula 2
        cv2.putText(img,str(int(moleculas[1].angle)),(3*tam//4,55),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255))
        update(moleculas,ka,a_eq)
        mostrarMoleculas(moleculas,wIm)


In [ ]:
if __name__=='__main__':
    #carga,angulo,cte
    main(5,104.5,7)

Image(value=b'')